In [31]:
from datasets import load_dataset

ds = load_dataset("UCSC-VLAA/Recap-DataComp-1B", split="preview",streaming=True)

In [32]:
ds

IterableDataset({
    features: ['url', 're_caption', 'org_caption', 'sha256', 'key', 're_clip_score', 'org_clip_score', 're_length', 'org_length', 're_gpt4v_score', 'org_gpt4v_score'],
    n_shards: 1
})

In [36]:
count = 0
for example in ds:
    print(example)
    count += 1
    if count > 0:
        break

{'url': 'https://images-na.ssl-images-amazon.com/images/I/41kzIaKFE+L._AC._SR180,230.jpg', 're_caption': 'A modern coffee machine with a digital display and two white coffee cups filled with coffee is shown. The machine has a stainless steel finish and is accompanied by a milk frothing pitcher with a white liquid inside. The coffee machine is placed on a surface with a white background.', 'org_caption': 'Saeco Xelsis Automatic Espresso Machine, SM7685/04, Stainless Steel', 'sha256': '501ac5e3066ff4cdad97dafd8916a65673b5e6d961c6e31e59dbeadd7fd7e3c7', 'key': '000000000008', 're_clip_score': 1.0, 'org_clip_score': 5.960464477539064e-08, 're_length': 50, 'org_length': 8, 're_gpt4v_score': 5, 'org_gpt4v_score': 5}


In [ ]:
# 'url'
# 're_caption'
# 'org_caption'
# 'key'
# 're_length'
# 'org_length'